In [ ]:
# Import neded packages

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
import numpy as np
import matplotlib.pyplot as plt
import matplotlib


# #%matplotlib widget
# %matplotlib inline


In [ ]:
# We move our tensor to the GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

In [ ]:
# Defined the DGM network


#__________________________ The class that defines the DGM layer ______________________________

class DGM_layer(nn.Module):
    """
        The parametres:
                        d: dimension of the space domain
                        M: number of units in each layer
                  """

    def __init__(self,d,M):
        super(DGM_layer, self).__init__()
        self.Uz = nn.Linear(d, M, bias=False)
        self.Wzbz = nn.Linear(M, M)
        self.Ug = nn.Linear(d, M, bias=False)
        self.Wgbg = nn.Linear(M, M)
        self.Ur = nn.Linear(d, M, bias=False)
        self.Wrbr = nn.Linear(M, M)
        self.Uh = nn.Linear(d, M, bias=False)
        self.Whbh = nn.Linear(M, M)
        self.onesTens = torch.ones(M).to(device)

    def activation(self, x):
        return torch.tanh(x)
        #return torch.sigmoid(x)
        #return x * torch.sigmoid(x)
        #return torch.relu(x)
        #return torch.cos(x)

    def forward(self, xt, prevS):
        Z = self.activation(self.Uz(xt) + self.Wzbz(prevS))
        G = self.activation(self.Ug(xt) + self.Wgbg(prevS))
        R = self.activation(self.Ur(xt) + self.Wrbr(prevS))
        SR = prevS * R
        H = self.activation(self.Uh(xt) + self.Whbh(SR))
        return (self.onesTens - G) * H + Z * prevS


#__________________________ The class that defines the DGM network ______________________________


class DGM_Net(nn.Module):
    """
        The parametres:
                        d: dimension of the space domain
                        M: number of units in each layer
                        L: number of DGM layers
                        X: the vector of  spatial data
                        t: the vector of time data
                  """

    def __init__(self, d, M, L):
        super(DGM_Net, self).__init__()
        self.initial_layer = nn.Linear(d, M)
        self.middle_layers = nn.ModuleList([DGM_layer(d, M) for i in range(L)])
        self.final_layer = nn.Linear(M, 1)

    def activation(self, x):
        return torch.tanh(x)
        #return torch.sigmoid(x)
        #return x * torch.sigmoid(x)
        #return torch.relu(x)
        #return torch.cos(x)

    def forward(self, X):
        S = self.activation(self.initial_layer(X))
        for i, DGMlayer in enumerate(self.middle_layers):
            S = DGMlayer(X, S)
        return self.final_layer(S)


In [ ]:
#_______________ hyperparameters __________________________________

dim = 2                          # imension of the space domain
M = 20                           # number of units in each layer
L = 3                            # number of DGM layers
num_eps = 50000                  # number of epochs totale

mini_batch_size = 1000

mini_batch_size_bdry = 250
x_low =  0.0
x_high = 1.0               # domain dimensions
pi = np.pi

area = np.abs(x_high - x_low)**dim    # domain measure for using the Monte Carlo approximation


# used in calculating boundary loss
ones = torch.ones((mini_batch_size_bdry, 1))
zeros = torch.zeros((mini_batch_size_bdry, 1))

In [ ]:
#___________________ the analytical solution __________________________________________________________

def U_exat(X):
    x = X[:,0].reshape(-1,1)
    y = X[:,1].reshape(-1,1)
    return torch.cos(pi*x**(2.5)) + torch.cos(pi*y**(2.5))

#___________________ the second member ________________________________________________________________

def f(x):
    u_output = U_exat(x)
    u_grad = torch.autograd.grad(u_output, x, grad_outputs=torch.ones_like(u_output), create_graph=True)[0]
    Δu = 0.0
    for i in range(dim):
        Δu += (torch.autograd.grad(u_grad[:,i], x, grad_outputs=torch.ones_like(u_grad[:,i]), create_graph=True)[0][:,i]).reshape(-1,1)
    return -Δu + pi**2 * u_output

In [ ]:
#_______________ Mean errors ____________________________________

def absulat_ME(Uext,Upred):
    return  torch.sqrt(((Uext - Upred)**2).mean())

def relative_E(Uext,Upred):
    return torch.sqrt(((Uext - Upred)**2).mean()/((Uext)**2).mean())*100

In [ ]:
# Define the learning rate schedule based on the number of epochs
def lr_lambda(epoch):
    if epoch <= 5000:
        return 1e-2
    elif 5000 < epoch <= 10000:
        return 5e-3
    elif 10000 < epoch <= 20000:
        return 1e-3
    elif 20000 < epoch <= 30000:
        return 5e-4
    elif 30000 < epoch <= 40000:
        return 1e-4
    elif 40000 < epoch <= 45000:
        return 5e-5
    else:
        return 1e-5

In [ ]:
#________________________ Training __________________________________________________________________________

u = DGM_Net(dim,M,L).to(device)   #  The network that will approach the solution


x_sampler = torch.distributions.uniform.Uniform(x_low, x_high)    #   to obtain the spatial data

# Create the optimizer and a LambdaLR scheduler with the defined learning rate schedule
optimizer = optim.Adam(u.parameters(),lr = 1)                   # Define the optimizer

scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)  # to adjust learning rate


loss_train = np.zeros(num_eps)             # loss of taining inside the domaine
relative_E_losses = np.zeros(num_eps)              # losses initialization

print('\n Using {} device'.format(device))
for ep in range(num_eps):


    x = x_sampler.sample((mini_batch_size, dim))
    x_bndr = x_sampler.sample((mini_batch_size_bdry, 1))

    x = torch.Tensor(x).to(device)
    x.requires_grad_()

    # evaluate forward pass, compute derivatives of network with respect to x
    u_output = u(x)
    u_grad = torch.autograd.grad(u_output, x, grad_outputs=torch.ones_like(u_output), create_graph=True)[0]

    Δu = 0.0
    for i in range(dim):
        Δu += (torch.autograd.grad(u_grad[:,i], x, grad_outputs=torch.ones_like(u_grad[:,i]), create_graph=True)[0][:,i]).reshape(-1,1)

    L1 = area * torch.mean(torch.pow((-Δu + pi**2 *u_output  - f(x)), 2))

    x_bry_1 = torch.cat((zeros,x_bndr),1).to(device)

    x_bry_1.requires_grad_()
    u_bry_1 = u(x_bry_1)
    u_grad_bry_1 = torch.autograd.grad(u_bry_1, x_bry_1, grad_outputs=torch.ones_like(u_bry_1), create_graph=True)[0][:,0]
    L2 = torch.mean(torch.pow(u_grad_bry_1, 2))

    x_bry_2 = torch.cat((ones,x_bndr),1).to(device)
    x_bry_2.requires_grad_()
    u_bry_2 = u(x_bry_2)
    u_grad_bry_2 = torch.autograd.grad(u_bry_2, x_bry_2, grad_outputs=torch.ones_like(u_bry_2), create_graph=True)[0][:,0]
    L3 = torch.mean(torch.pow(u_grad_bry_2, 2))

    y_bry_1 = torch.cat((x_bndr,zeros),1).to(device)
    y_bry_1.requires_grad_()
    u_bry_3 = u(y_bry_1)
    u_grad_bry_3 = torch.autograd.grad(u_bry_3, y_bry_1, grad_outputs=torch.ones_like(u_bry_3), create_graph=True)[0][:,1]
    L4 = torch.mean(torch.pow(u_grad_bry_3, 2))

    y_bry_2 = torch.cat((x_bndr,ones),1).to(device)
    y_bry_2.requires_grad_()
    u_bry_4 = u(y_bry_2)
    u_grad_bry_4 = torch.autograd.grad(u_bry_4, y_bry_2, grad_outputs=torch.ones_like(u_bry_4), create_graph=True)[0][:,1]
    L5 = torch.mean(torch.pow(u_grad_bry_4, 2))



    Loss = L1 + L2 + L3 + L4 + L5

    relative_E_losses[ep] =  float(relative_E( U_exat(x) ,u_output).item())

    optimizer.zero_grad()
    Loss.backward()            # compute derivative of loss with respect to network parameters
    optimizer.step()           # update network parameters with ADAM

    # Update the learning rate
    scheduler.step(ep)  # Pass the current epoch to the scheduler

    # Display the current learning rate
    current_lr = optimizer.param_groups[0]['lr']

    loss_train[ep] = float(L1.item())
    if ep % 100 == 99:
        print("Epoch %d - L_r:%f  loss_int : %.2f - REN : %.2f"%(ep, current_lr, loss_train[ep], relative_E_losses[ep]),"%")
        #Ploting(x,x_bndr, u)



Epoch 14199 - L_r:0.001000  loss_int : 0.00 - REN : 0.03 %
Epoch 14299 - L_r:0.001000  loss_int : 0.00 - REN : 0.04 %
Epoch 14399 - L_r:0.001000  loss_int : 0.01 - REN : 0.06 %
Epoch 14499 - L_r:0.001000  loss_int : 0.00 - REN : 0.03 %
Epoch 14599 - L_r:0.001000  loss_int : 0.02 - REN : 0.24 %
Epoch 14699 - L_r:0.001000  loss_int : 0.00 - REN : 0.05 %
Epoch 14799 - L_r:0.001000  loss_int : 0.00 - REN : 0.04 %
Epoch 14899 - L_r:0.001000  loss_int : 0.02 - REN : 0.12 %
Epoch 14999 - L_r:0.001000  loss_int : 0.00 - REN : 0.02 %
Epoch 15099 - L_r:0.001000  loss_int : 0.00 - REN : 0.01 %
Epoch 15199 - L_r:0.001000  loss_int : 0.00 - REN : 0.01 %
Epoch 15299 - L_r:0.001000  loss_int : 0.00 - REN : 0.01 %
Epoch 15399 - L_r:0.001000  loss_int : 0.00 - REN : 0.01 %
Epoch 15499 - L_r:0.001000  loss_int : 0.00 - REN : 0.01 %
Epoch 15599 - L_r:0.001000  loss_int : 0.00 - REN : 0.02 %
Epoch 15699 - L_r:0.001000  loss_int : 0.01 - REN : 0.06 %
Epoch 15799 - L_r:0.001000  loss_int : 0.00 - REN : 0.07

Epoch 28099 - L_r:0.000500  loss_int : 0.00 - REN : 0.03 %
Epoch 28199 - L_r:0.000500  loss_int : 0.00 - REN : 0.02 %
Epoch 28299 - L_r:0.000500  loss_int : 0.00 - REN : 0.01 %
Epoch 28399 - L_r:0.000500  loss_int : 0.00 - REN : 0.07 %
Epoch 28499 - L_r:0.000500  loss_int : 0.00 - REN : 0.03 %
Epoch 28599 - L_r:0.000500  loss_int : 0.00 - REN : 0.02 %
Epoch 28699 - L_r:0.000500  loss_int : 0.00 - REN : 0.01 %
Epoch 28799 - L_r:0.000500  loss_int : 0.00 - REN : 0.01 %
Epoch 28899 - L_r:0.000500  loss_int : 0.00 - REN : 0.01 %
Epoch 28999 - L_r:0.000500  loss_int : 0.00 - REN : 0.02 %
Epoch 29099 - L_r:0.000500  loss_int : 0.00 - REN : 0.02 %
Epoch 29199 - L_r:0.000500  loss_int : 0.00 - REN : 0.01 %
Epoch 29299 - L_r:0.000500  loss_int : 0.00 - REN : 0.01 %
Epoch 29399 - L_r:0.000500  loss_int : 0.00 - REN : 0.01 %
Epoch 29499 - L_r:0.000500  loss_int : 0.00 - REN : 0.06 %
Epoch 29599 - L_r:0.000500  loss_int : 0.00 - REN : 0.01 %
Epoch 29699 - L_r:0.000500  loss_int : 0.00 - REN : 0.01

Epoch 41999 - L_r:0.000050  loss_int : 0.00 - REN : 0.01 %
Epoch 42099 - L_r:0.000050  loss_int : 0.00 - REN : 0.00 %
Epoch 42199 - L_r:0.000050  loss_int : 0.00 - REN : 0.01 %
Epoch 42299 - L_r:0.000050  loss_int : 0.00 - REN : 0.00 %
Epoch 42399 - L_r:0.000050  loss_int : 0.00 - REN : 0.01 %
Epoch 42499 - L_r:0.000050  loss_int : 0.00 - REN : 0.01 %
Epoch 42599 - L_r:0.000050  loss_int : 0.00 - REN : 0.00 %
Epoch 42699 - L_r:0.000050  loss_int : 0.00 - REN : 0.01 %
Epoch 42799 - L_r:0.000050  loss_int : 0.00 - REN : 0.00 %
Epoch 42899 - L_r:0.000050  loss_int : 0.00 - REN : 0.00 %
Epoch 42999 - L_r:0.000050  loss_int : 0.00 - REN : 0.00 %
Epoch 43099 - L_r:0.000050  loss_int : 0.00 - REN : 0.00 %
Epoch 43199 - L_r:0.000050  loss_int : 0.00 - REN : 0.00 %
Epoch 43299 - L_r:0.000050  loss_int : 0.00 - REN : 0.00 %
Epoch 43399 - L_r:0.000050  loss_int : 0.00 - REN : 0.00 %
Epoch 43499 - L_r:0.000050  loss_int : 0.00 - REN : 0.00 %
Epoch 43599 - L_r:0.000050  loss_int : 0.00 - REN : 0.01

In [ ]:

#_______________________________________________________________________________

grid_nums = 500
x_test = torch.distributions.uniform.Uniform(x_low, x_high).sample((grid_nums, dim)).to(device)
with torch.no_grad():
    U = u(x_test).detach()

U_ext =  U_exat(x_test)

print("The Mean Absulat Error: {}".format(absulat_ME(U_ext,U)))
print("The Relative Error: {} %".format(relative_E(U_ext,U)))

In [ ]:
MSEabs = 0
MSErlf = 0
N = 100
for compt in range(N):
    x_test = torch.distributions.uniform.Uniform(x_low, x_high).sample((grid_nums, dim)).to(device)
    U = u(x_test)
    U_ext =  U_exat(x_test)

    MSEabs+= absulat_ME(U_ext,U)
    MSErlf+= relative_E(U_ext,U)

print("The averag of Mean Absulat Error: {}".format(MSEabs/N))
print("The averag of Relative Error: {} %".format(MSErlf/N))

In [ ]:
# #_______________________ Save Model _____________________________________

# torch.save(u.state_dict(), "Model_DGM_Problem_with_less_regular_solution_1000") # to save weights
# np.save("loss_DGM_Problem_with_less_regular_solution_1000" ,Losse)                # to save Losse


In [ ]:
# # # # #_______________________ load Model _____________________________________

# u_IS = DGM_Net(dim,M,L).to(device)
# state_dict = torch.load( "Model_IS_DGM_Poisson_special_50000_2.pth")
# u_IS.load_state_dict(state_dict)
# losse_IS = np.load("loss_IS_DGM_Poisson_special_50000_2.npy")

In [ ]:
# # #_______________ Ploting results __________________________________


# grid_nums = 50

# x_grid = torch.linspace(-1.0,1.0,grid_nums)
# XX, YY = torch.meshgrid(x_grid,x_grid)
# X = torch.reshape(XX, (-1,1))
# Y = torch.reshape(YY, (-1,1))
# XY = torch.cat((X,Y),1).to(device)
# with torch.no_grad():
#     U = u_IS(XY)

# UU = torch.reshape(U, (grid_nums,grid_nums)).detach().cpu().numpy()
# U_ext =  U_exat(XY)
# U_ext = torch.reshape(U_ext , (grid_nums,grid_nums)).detach().cpu().numpy()

# fig = plt.figure()
# ax = fig.add_subplot(projection='3d')
# ax.plot_surface(XX.numpy(),YY.numpy(), UU )


In [ ]:
# grid_nums = 500
# x_test = torch.distributions.uniform.Uniform(x_low, x_high).sample((grid_nums, dim)).to(device)
# with torch.no_grad():
#     U = u(x_test).detach()
#     U_IS = u_IS(x_test).detach()


# U_ext =  U_exat(x_test)

# print("MAE_standard_DGM = {} | MAE_Proposed method = {}".format(absulat_ME(U_ext,U), absulat_ME(U_ext,U_IS)))

# print("MRE_standard_DGM = {}% | MRE_Proposed method = {}%".format(relative_E(U_ext,U), relative_E(U_ext,U_IS)))

In [ ]:
# Losse_IS  = np.log(losse_IS)
# Losse_standard  = np.log(Losse)

# x_error = np.linspace(0,50000, 50000).reshape(-1,1)
# plt.plot(x_error, Losse_IS,  label='Losse_IS')
# plt.plot(x_error, Losse_standard, label='Losse_standard')
# plt.legend()
# plt.show